In [11]:
import pandas as pd
import numpy as np
from feature_engeneering import modify_df
from feature_engeneering import drop_columns
from feature_engeneering import create_differences_columns

In [3]:
df = pd.read_csv('../data/output/in_time_cleaned.csv')

In [4]:
df_modified = modify_df(df)

/Users/kacpergruca/Documents/Projekty/ING_Lions_Den_2024_final/RiskBusters_Codes/feature_engeneering.py:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Birth_date'] = pd.to_datetime(df['Birth_date'])


In [5]:
columns = df_modified.columns
columns

Index(['Ref_month', 'Customer_id', 'Birth_date', 'No_dependants',
       'Time_in_address', 'Time_in_current_job', 'Credit_cards', 'Debit_cards',
       'Active_accounts', 'Oldest_account_date',
       ...
       'Avg_inc_transactions_H8', 'Avg_out_transactions_H8',
       'Avg_inc_transactions_H9', 'Avg_out_transactions_H9',
       'Avg_inc_transactions_H10', 'Avg_out_transactions_H10',
       'Avg_inc_transactions_H11', 'Avg_out_transactions_H11',
       'Avg_inc_transactions_H12', 'Avg_out_transactions_H12'],
      dtype='object', length=405)

In [6]:
# write colnames of df_modified to dataframe
df_colnames = pd.DataFrame(columns)
df_colnames

,0
0,Ref_month
1,Customer_id
2,Birth_date
3,No_dependants
4,Time_in_address
...,...
400,Avg_out_transactions_H10
401,Avg_inc_transactions_H11
402,Avg_out_transactions_H11
403,Avg_inc_transactions_H12


## delete columns

In [7]:
columns_to_delete_H = ['inc_transactions_amt_H', 'out_transactions_amt_H', 'overdue_term_loan_H', 
                       'overdue_credit_card_H', 'overdue_mortgage_H', 'Income_H',
                       'current_amount_balance_H', 'savings_amount_balance_H', 'inc_transactions_amt_H',
                        'Savings_amount_balance_col_H', 'Current_amount_balance_col_H',
                        'Overdue_term_loan_H', 'Overdue_credit_card_H', 'Overdue_mortgage_H',
                        'inc_transactions_H', 'out_transactions_H', 'inc_transactions_amt_H', 'out_transactions_amt_H',
                        'utilized_limit_in_revolving_loans_H', 'limit_in_revolving_loans_H']

columns_to_delete = ['Current_installment', 'Birth_date', 'Ref_month',
                     'Contract_end_date', 'Oldest_account_date']

In [8]:
df_tes_function = df_modified.copy()

In [9]:
# list of columns which has _H in the name in df_tes 

columns_H = [col for col in df_tes_function.columns if '_H0' in col]

# delete last letter from oevery column in columns_H
columns_H = [col[:-1] for col in columns_H]

In [10]:
columns_H

['DPD_term_loan_H',
 'DPD_credit_card_H',
 'DPD_mortgage_H',
 'Default_flag_H',
 'Os_term_loan_H',
 'Os_credit_card_H',
 'Os_mortgage_H',
 'Overdue_term_loan_H',
 'Overdue_credit_card_H',
 'Overdue_mortgage_H',
 'limit_in_revolving_loans_H',
 'utilized_limit_in_revolving_loans_H',
 'Payments_term_loan_H',
 'Payments_credit_card_H',
 'Payments_mortgage_H',
 'Income_H',
 'Current_amount_balance_H',
 'Savings_amount_balance_H',
 'inc_transactions_H',
 'out_transactions_H',
 'inc_transactions_amt_H',
 'out_transactions_amt_H',
 'DTI_H',
 'Overdue_to_Income_Ratio_H',
 'Financial_Liquidity_Ratio_H',
 'Financial_stability_index_H',
 'Overdue_aggregation_H',
 'Avg_inc_transactions_H',
 'Avg_out_transactions_H']

In [66]:
df_tes_function = create_differences_columns(df_tes_function, columns_H)

In [68]:
# drop columns with _H in the name

def drop_columns_after_transformation(df):
    for colname in columns_H:
        for i in range(13):
                colname2 = colname + str(i)
                if colname2 in df.columns:
                    df = df.drop(colname2, axis=1)
    return df

In [69]:
df_tes_function = drop_columns_after_transformation(df_tes_function)

In [70]:
columns = df_tes_function.columns
columns = pd.DataFrame(columns)
columns

,0
0,Customer_id
1,No_dependants
2,Time_in_address
3,Time_in_current_job
4,Credit_cards
...,...
88,Avg_inc_transactions_H0_H6
89,Avg_inc_transactions_H0_H12
90,Avg_out_transactions_H0_H3
91,Avg_out_transactions_H0_H6


In [59]:
nulls = df_tes_function.isna().sum()
nulls[nulls > 0]

Overdue_to_Income_Ratio_H0             1
Financial_stability_index_H0        1533
Financial_stability_index_H1        1481
Financial_stability_index_H2        1571
Financial_stability_index_H3        1565
Financial_stability_index_H4        1567
Financial_stability_index_H5        1540
Financial_stability_index_H6        1537
Financial_stability_index_H7        1532
Financial_stability_index_H8        1509
Financial_stability_index_H9        1514
Financial_stability_index_H10       1516
Financial_stability_index_H11       1569
Financial_stability_index_H12       1536
Overdue_to_Income_Ratio_H0_H3          1
Overdue_to_Income_Ratio_H0_H6          1
Overdue_to_Income_Ratio_H0_H12         1
Financial_stability_index_H0_H3     3090
Financial_stability_index_H0_H6     3061
Financial_stability_index_H0_H12    3064
dtype: int64

In [60]:
# drop all NaN values
df_tes_function = df_tes_function.dropna()

## clients_age

In [8]:
# change the type of Birth_date to datetime
df['Birth_date'] = pd.to_datetime(df['Birth_date'])

In [9]:
# add a column clients_age with the age of the client by substracting the birth_date from the current date
current_date = pd.to_datetime('2024-04-03')
df['clients_age'] = (current_date - df['Birth_date']).astype('<m8[Y]').astype(int)

## Create difference columns

Approach to new variables:

1. We crate 12 new variables for every Hx
2. We calculate for every column:

(H0 - H3) / H0
(H0 - H6) / H0
(H0 - H12) / H0

## Create DTI

(inc_transactions_amt - out_transactions_amt) / current_installment

In [ ]:
columns_to_delete_H = ['inc_transactions_amt_H', 'out_transactions_amt_H']
columns_to_delete = ['Current_installment', 'Birth_date']